In [62]:
import import_ipynb
from model import UNet3D
from data_loader import BrainTumorDataset

importing Jupyter notebook from model.ipynb
importing Jupyter notebook from data_loader.ipynb


In [68]:
from torch.utils.data import DataLoader

TRAIN_OUTPUT_IMAGE_PATH = '.\BraTS20\BraTS2020_TrainingData\input_data_4channels\images\\'
TRAIN_OUTPUT_MASK_PATH = '.\BraTS20\BraTS2020_TrainingData\input_data_4channels\masks\\'
VAL_OUTPUT_IMAGE_PATH = '.\BraTS20\BraTS2020_TrainingData\input_data_4channels\images\\'
VAL_OUTPUT_MASK_PATH = '.\BraTS20\BraTS2020_TrainingData\input_data_4channels\masks\\'

train_dataset = BrainTumorDataset(image_dir, mask_dir)

batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [54]:
import numpy as np
wt0, wt1, wt2, wt3 = 0.25,0.25,0.25,0.25
import segmentation_models_3D as sm
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [64]:
import torch.optim as optim

model = model = UNet3D(4, 4)  
LR = 0.0001
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
num_epochs = 100

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (images, masks) in enumerate(train_loader):
        images = images.cuda()
        masks = masks.cuda()
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = total_loss(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 10 == 9:  # Print every 10 batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}')
            running_loss = 0.0

print('Finished Training')
torch.save(model.state_dict(), '3d_unet_brats.pth')
